In [1]:
import polars as pl

In [2]:
df = pl.read_csv("weather.csv", has_header= True)

In [3]:
country = df.select(
                    pl.col(["Station.State", "Data.Temperature.Max Temp", "Data.Temperature.Min Temp"]),
                    (pl.col("Data.Temperature.Max Temp") + pl.col("Data.Temperature.Min Temp")).alias("Total max & min"))

q = (
     country.lazy()
     .group_by("Station.State")
     .agg(
          pl.mean("Total max & min").alias("Total max & min")
     )
     .sort("Total max & min", descending= True)
)

q.collect().head(5)

Station.State,Total max & min
str,f64
"""Puerto Rico""",162.188679
"""Hawaii""",152.320755
"""Florida""",147.382075
"""Louisiana""",138.467433
"""Texas""",135.86478


In [4]:
df_2 = df.select(
                 pl.col("Station.State"),
                 pl.col("Station.Location"),
                 pl.col("Data.Temperature.Avg Temp"),
                 pl.col("Data.Temperature.Avg Temp").sum().over("Station.State").alias("country avg temp")
)

df_2.head(5)

Station.State,Station.Location,Data.Temperature.Avg Temp,country avg temp
str,str,i64,i64
"""Alabama""","""Birmingham, AL…",39,14104
"""Alabama""","""Huntsville, AL…",39,14104
"""Alabama""","""Mobile, AL""",46,14104
"""Alabama""","""Montgomery, AL…",45,14104
"""Alaska""","""Anchorage, AK""",34,70750


In [5]:
df_2.select(pl.col(["Station.State", "Data.Temperature.Avg Temp"]),
            ((pl.col("Data.Temperature.Avg Temp") / pl.col("country avg temp")) * 100).alias("avg")
).head(5)

Station.State,Data.Temperature.Avg Temp,avg
str,i64,f64
"""Alabama""",39,0.276517
"""Alabama""",39,0.276517
"""Alabama""",46,0.326149
"""Alabama""",45,0.319058
"""Alaska""",34,0.048057
